In [1]:
# UNCOMMENT this in colab

# ##### xiyah #####
# # mount google drive
# # since I uploaded all data into google drive.
# from google.colab import drive
# drive.mount('/content/drive')


# # install librosa
# # after installation can comment out
# # "-q > /dev/null" helps hide the install message
# !pip install librosa -q > /dev/null
# ##### xiyah #####

Mounted at /content/drive


In [1]:
# import packages
import os
import librosa
import glob
import pandas as pd
import numpy as np

# used for storing features into files
import pickle

# used for calculating statistical features
from scipy.stats import skew, kurtosis

# imports for model
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset

## Drop Useless Data (**You can skip this step, since all the data after drops are saved**)

drop those data which are:   

- not a 1d vector or a scalar
- not useful for training("xs", "srs")

**You can skip this step, since all the data after drops are saved**

In [4]:
# load .npy
q1_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/all_features_q1_mod.npy"
q2_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/all_features_q2_mod.npy"
q3_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/all_features_q3_mod.npy"
q4_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/all_features_q4_mod.npy"

# np.load
data_q1 = np.load(q1_path, allow_pickle=True)
data_q2 = np.load(q2_path, allow_pickle=True)
data_q3 = np.load(q3_path, allow_pickle=True)
data_q4 = np.load(q4_path, allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/all_features_q2_mod.npy'

In [ ]:
# define a columns' names variable
df_columns = ['xs', 'srs',
              'chroma_stft_demos', 'chroma_cqt_demos', 'chroma_vqt_demos', 'chroma_cens_demos', \
              'melspectrogram_demos', 'mfcc_demos', 'rms_demos', \
              'spectral_centroid_demos', 'spectral_bandwidth_demos', 'spectral_contrast_demos', 'spectral_flatness_demos', 'spectral_rolloff_demos', \
              'tonnetz_demos', 'zero_crossing_rate_demos', 'tempo', 'tempogram_ratio', 'labels', \
              'chroma_stft_mean', 'chroma_stft_std', 'chroma_stft_sum', 'chroma_stft_skew', 'chroma_stft_kurtosis', \
              'chroma_cqt_mean', 'chroma_cqt_std', 'chroma_cqt_sum', 'chroma_cqt_skew', 'chroma_cqt_kurtosis', \
              'chroma_vqt_mean', 'chroma_vqt_std', 'chroma_vqt_sum', 'chroma_vqt_skew', 'chroma_vqt_kurtosis', \
              'melspectrogram_mean', 'melspectrogram_std', 'melspectrogram_min', 'melspectrogram_max', 'melspectrogram_sum', 'melspectrogram_skew', 'melspectrogram_kurtosis', \
              'mfcc_mean', 'mfcc_std', 'mfcc_min', 'mfcc_max', 'mfcc_skew', 'mfcc_kurtosis', 'mfcc_median', \
              'rms_mean', 'rms_std', 'rms_min', 'rms_max', 'rms_skew', 'rms_kurtosis', \
              'spectral_centroid_mean', 'spectral_centroid_std', 'spectral_centroid_min', 'spectral_centroid_max', 'spectral_centroid_skew', 'spectral_centroid_kurtosis', \
              'spectral_bandwidth_mean', 'spectral_bandwidth_std', 'spectral_bandwidth_min', 'spectral_bandwidth_max', 'spectral_bandwidth_skew', 'spectral_bandwidth_kurtosis', \
              'spectral_contrast_mean', 'spectral_contrast_std', 'spectral_contrast_min', 'spectral_contrast_max', 'spectral_contrast_skew', 'spectral_contrast_kurtosis', 'spectral_contrast_range', \
              'spectral_flatness_mean', 'spectral_flatness_std', 'spectral_flatness_min', 'spectral_flatness_max', 'spectral_flatness_skew', 'spectral_flatness_kurtosis', 'spectral_flatness_range', \
              'spectral_rolloff_mean', 'spectral_rolloff_std', 'spectral_rolloff_min', 'spectral_rolloff_max', 'spectral_rolloff_skew', 'spectral_rolloff_kurtosis', 'spectral_rolloff_range', \
              'tonnetz_mean', 'tonnetz_std', 'tonnetz_min', 'tonnetz_max', 'tonnetz_skew', 'tonnetz_kurtosis', 'tonnetz_range', \
              'zero_crossing_rate_mean', 'zero_crossing_rate_std', 'zero_crossing_rate_min', 'zero_crossing_rate_max', 'zero_crossing_rate_skew', 'zero_crossing_rate_kurtosis', 'zero_crossing_rate_range']

In [ ]:
### Xiyah ###
# convert all .npy to pd.DataFrame
df_q1 = pd.DataFrame(data_q1, columns=df_columns)
df_q2 = pd.DataFrame(data_q2, columns=df_columns)
df_q3 = pd.DataFrame(data_q3, columns=df_columns)
df_q4 = pd.DataFrame(data_q4, columns=df_columns)
#df_q1.head()
### Xiyah ###

In [ ]:
### Xiyah ###
# Get shape of all columns
# Function to get the shape of each element
def get_shape(element):
    # Check if the element is a NumPy array
    if hasattr(element, 'shape'):
        return element.shape
    # Check if the element is iterable (like a list) but not a string
    elif hasattr(element, '__iter__') and not isinstance(element, str):
        return len(element),
    # Handle non-iterable objects (like integers)
    else:
        return ()  # Return an empty tuple or another placeholder indicating non-iterable objects

# Apply `get_shape` to only the first row
shape_info_q1 = df_q1.iloc[0].apply(get_shape)

#print(shape_info_q1)

# Filter columns where the shape is not of length 0 or 1, indicating they are not scalars or 1D vectors
cols_to_drop = shape_info_q1[shape_info_q1.apply(len) > 1].index.tolist()

print(f"columns to drop: {cols_to_drop}")
### Xiyah ###

columns to drop: ['chroma_stft_demos', 'chroma_cqt_demos', 'chroma_vqt_demos', 'chroma_cens_demos', 'melspectrogram_demos', 'mfcc_demos', 'rms_demos', 'spectral_centroid_demos', 'spectral_bandwidth_demos', 'spectral_contrast_demos', 'spectral_flatness_demos', 'spectral_rolloff_demos', 'tonnetz_demos', 'zero_crossing_rate_demos', 'tempogram_ratio']


In [ ]:
### Xiyah ###
# drop all columns that are not 1d or scalars: we already have their mathematical features
df_q1 = df_q1.drop(columns=cols_to_drop)
df_q2 = df_q2.drop(columns=cols_to_drop)
df_q3 = df_q3.drop(columns=cols_to_drop)
df_q4 = df_q4.drop(columns=cols_to_drop)

In [ ]:
# After inspection, we will also drop xs and srs, they are only useful for featurization
df_q1 = df_q1.drop(columns=['xs', 'srs'])
df_q2 = df_q2.drop(columns=['xs', 'srs'])
df_q3 = df_q3.drop(columns=['xs', 'srs'])
df_q4 = df_q4.drop(columns=['xs', 'srs'])

In [ ]:
# re-order columns, move labels as the last column
# Get a list of all column names except 'labels'
cols_no_labels_q1 = [col for col in df_q1.columns if col != 'labels']
cols_no_labels_q2 = [col for col in df_q2.columns if col != 'labels']
cols_no_labels_q3 = [col for col in df_q3.columns if col != 'labels']
cols_no_labels_q4 = [col for col in df_q4.columns if col != 'labels']

# Append 'labels' to the end of the column list
new_cols_q1 = cols_no_labels_q1 + ['labels']
new_cols_q2 = cols_no_labels_q2 + ['labels']
new_cols_q3 = cols_no_labels_q3 + ['labels']
new_cols_q4 = cols_no_labels_q4 + ['labels']

# Reorder the DataFrame columns
df_q1 = df_q1[new_cols_q1]
df_q2 = df_q2[new_cols_q2]
df_q3 = df_q3[new_cols_q3]
df_q4 = df_q4[new_cols_q4]

In [ ]:
print(f"df_q1.shape: {df_q1.shape}")
print(f"df_q2.shape: {df_q2.shape}")
print(f"df_q3.shape: {df_q3.shape}")
print(f"df_q4.shape: {df_q4.shape}")

df_q1.shape: (225, 84)
df_q2.shape: (225, 84)
df_q3.shape: (225, 84)
df_q4.shape: (225, 84)


In [ ]:
df_q1.head()

,tempo,chroma_stft_mean,chroma_stft_std,chroma_stft_sum,chroma_stft_skew,chroma_stft_kurtosis,chroma_cqt_mean,chroma_cqt_std,chroma_cqt_sum,chroma_cqt_skew,...,tonnetz_kurtosis,tonnetz_range,zero_crossing_rate_mean,zero_crossing_rate_std,zero_crossing_rate_min,zero_crossing_rate_max,zero_crossing_rate_skew,zero_crossing_rate_kurtosis,zero_crossing_rate_range,labels
0,[129.19921875],"[0.3402438, 0.4108848, 0.32207942, 0.43924117,...","[0.25901443, 0.34396023, 0.24817161, 0.3390962...",5754.600098,"[0.99369633, 0.6889807, 1.1558719, 0.51363957,...","[0.18223047, -1.0339843, 0.7369144, -1.18541, ...","[0.37230334, 0.59613794, 0.349376, 0.5221062, ...","[0.1650009, 0.28621086, 0.17438404, 0.24190375...",7482.439453,"[1.1793867, 0.19034995, 1.4648391, 0.5109394, ...",...,"[-0.2676932002114798, -0.2611242959606561, -0....","[0.39503816324089674, 0.44382569880070666, 0.6...",[0.1439822635135135],[0.06052925895431856],[0.03125],[0.564453125],[2.299913592613744],[9.972581240443636],[0.533203125],1
1,[161.4990234375],"[0.36104006, 0.2428113, 0.33913192, 0.41598046...","[0.3057084, 0.21803722, 0.26099977, 0.31606582...",5868.901367,"[0.8817804, 1.4171929, 0.9687368, 0.77790874, ...","[-0.35459352, 1.3924265, 0.051932335, -0.83470...","[0.39986077, 0.30028376, 0.41042277, 0.5339542...","[0.2795176, 0.11410666, 0.1324839, 0.23995899,...",6873.734375,"[1.3378943, 1.0632907, 0.9806561, 0.91683173, ...",...,"[-0.4546138798763306, -0.5757695864820596, 0.0...","[0.42942833816117487, 0.4362426048761109, 0.52...",[0.10062665902509653],[0.06195558099185238],[0.0126953125],[0.53857421875],[2.4103148531571437],[10.632752706921927],[0.52587890625],1
2,[99.38401442307692],"[0.3350637, 0.4391525, 0.3131803, 0.36769086, ...","[0.23067027, 0.34413788, 0.2166038, 0.3000164,...",5885.837891,"[0.63521445, 0.6002824, 0.6111119, 0.9701896, ...","[-0.44197345, -1.1731204, -0.6007376, -0.27134...","[0.2838195, 0.55256397, 0.23260526, 0.35447538...","[0.13484542, 0.31740782, 0.123309836, 0.267685...",6406.210449,"[1.078106, 0.44249153, 1.7751677, 1.425022, 1....",...,"[0.07777779783759664, -0.66986980775311, -0.57...","[0.549337563990827, 0.45787447752394766, 0.692...",[0.06854375301640926],[0.03186446827069103],[0.01806640625],[0.21875],[1.0653329036995505],[1.313176804637732],[0.20068359375],1
3,[123.046875],"[0.20232584, 0.4100135, 0.2596639, 0.14443561,...","[0.15172842, 0.36189678, 0.26187456, 0.1126129...",4598.774902,"[1.5267997, 0.74306566, 1.7494311, 1.5250498, ...","[3.3818812, -1.0821086, 2.2402334, 3.302959, 1...","[0.19787204, 0.38152912, 0.2595981, 0.18471201...","[0.11222625, 0.2533469, 0.15406808, 0.10194606...",4871.200684,"[1.6630076, 1.2764502, 1.5215753, 1.4900329, 1...",...,"[-0.689955821707545, 0.011593890255965356, -0....","[0.6551601962522167, 0.8039702548013667, 0.817...",[0.0696255127895753],[0.016080018069265088],[0.02978515625],[0.1484375],[0.7492532964882583],[1.9720264141033086],[0.11865234375],1
4,[103.359375],"[0.20571218, 0.18657087, 0.284147, 0.23309906,...","[0.26805663, 0.24865834, 0.31335503, 0.2693379...",3841.712158,"[1.8727081, 1.9668415, 1.3176929, 1.7174534, 1...","[2.6374311, 3.2114024, 0.471771, 2.033866, 0.9...","[0.4677634, 0.34280092, 0.41840968, 0.42241988...","[0.29720858, 0.25228196, 0.27344728, 0.2717988...",6369.010254,"[0.54591733, 1.2141986, 0.8864644, 0.9038617, ...",...,"[2.0776852407507347, 1.3959966015028629, 0.106...","[0.9353934797999381, 0.8043349604683768, 1.031...",[0.09968026061776061],[0.039801127600770446],[0.025390625],[0.23291015625],[0.24549962817541723],[-0.5180193190257767],[0.20751953125],1


In [ ]:
# save to .npy after drop
df_q1_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/data_afterDrop_q1.npy"
df_q2_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/data_afterDrop_q2.npy"
df_q3_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/data_afterDrop_q3.npy"
df_q4_path = "/content/drive/MyDrive/EN.553.602DataMiningSpring2024/dataset/data_afterDrop_q4.npy"
# # to array
arr_q1 = df_q1.to_numpy()
arr_q2 = df_q2.to_numpy()
arr_q3 = df_q3.to_numpy()
arr_q4 = df_q4.to_numpy()
# # save  (225, 84) for each
np.save(df_q1_path, arr_q1, allow_pickle=True)
np.save(df_q2_path, arr_q2, allow_pickle=True)
np.save(df_q3_path, arr_q3, allow_pickle=True)
np.save(df_q4_path, arr_q4, allow_pickle=True)

In [ ]:
# try loading
# np.load
mod_data_q1 = np.load(df_q1_path, allow_pickle=True)
mod_data_q2 = np.load(df_q2_path, allow_pickle=True)
mod_data_q3 = np.load(df_q3_path, allow_pickle=True)
mod_data_q4 = np.load(df_q4_path, allow_pickle=True)

In [ ]:
print(list(df_q1.columns))

['tempo', 'chroma_stft_mean', 'chroma_stft_std', 'chroma_stft_sum', 'chroma_stft_skew', 'chroma_stft_kurtosis', 'chroma_cqt_mean', 'chroma_cqt_std', 'chroma_cqt_sum', 'chroma_cqt_skew', 'chroma_cqt_kurtosis', 'chroma_vqt_mean', 'chroma_vqt_std', 'chroma_vqt_sum', 'chroma_vqt_skew', 'chroma_vqt_kurtosis', 'melspectrogram_mean', 'melspectrogram_std', 'melspectrogram_min', 'melspectrogram_max', 'melspectrogram_sum', 'melspectrogram_skew', 'melspectrogram_kurtosis', 'mfcc_mean', 'mfcc_std', 'mfcc_min', 'mfcc_max', 'mfcc_skew', 'mfcc_kurtosis', 'mfcc_median', 'rms_mean', 'rms_std', 'rms_min', 'rms_max', 'rms_skew', 'rms_kurtosis', 'spectral_centroid_mean', 'spectral_centroid_std', 'spectral_centroid_min', 'spectral_centroid_max', 'spectral_centroid_skew', 'spectral_centroid_kurtosis', 'spectral_bandwidth_mean', 'spectral_bandwidth_std', 'spectral_bandwidth_min', 'spectral_bandwidth_max', 'spectral_bandwidth_skew', 'spectral_bandwidth_kurtosis', 'spectral_contrast_mean', 'spectral_contrast_s

In [ ]:
# assign columns names
cols = ['tempo', 'chroma_stft_mean', 'chroma_stft_std', 'chroma_stft_sum', 'chroma_stft_skew', 'chroma_stft_kurtosis',
        'chroma_cqt_mean', 'chroma_cqt_std', 'chroma_cqt_sum', 'chroma_cqt_skew', 'chroma_cqt_kurtosis',
        'chroma_vqt_mean', 'chroma_vqt_std', 'chroma_vqt_sum', 'chroma_vqt_skew', 'chroma_vqt_kurtosis',
        'melspectrogram_mean', 'melspectrogram_std', 'melspectrogram_min', 'melspectrogram_max', 'melspectrogram_sum', 'melspectrogram_skew', 'melspectrogram_kurtosis',
        'mfcc_mean', 'mfcc_std', 'mfcc_min', 'mfcc_max', 'mfcc_skew', 'mfcc_kurtosis', 'mfcc_median',
        'rms_mean', 'rms_std', 'rms_min', 'rms_max', 'rms_skew', 'rms_kurtosis',
        'spectral_centroid_mean', 'spectral_centroid_std', 'spectral_centroid_min', 'spectral_centroid_max', 'spectral_centroid_skew', 'spectral_centroid_kurtosis',
        'spectral_bandwidth_mean', 'spectral_bandwidth_std', 'spectral_bandwidth_min', 'spectral_bandwidth_max', 'spectral_bandwidth_skew', 'spectral_bandwidth_kurtosis',
        'spectral_contrast_mean', 'spectral_contrast_std', 'spectral_contrast_min', 'spectral_contrast_max', 'spectral_contrast_skew', 'spectral_contrast_kurtosis', 'spectral_contrast_range',
        'spectral_flatness_mean', 'spectral_flatness_std', 'spectral_flatness_min', 'spectral_flatness_max', 'spectral_flatness_skew', 'spectral_flatness_kurtosis', 'spectral_flatness_range',
        'spectral_rolloff_mean', 'spectral_rolloff_std', 'spectral_rolloff_min', 'spectral_rolloff_max', 'spectral_rolloff_skew', 'spectral_rolloff_kurtosis', 'spectral_rolloff_range',
        'tonnetz_mean', 'tonnetz_std', 'tonnetz_min', 'tonnetz_max', 'tonnetz_skew', 'tonnetz_kurtosis', 'tonnetz_range',
        'zero_crossing_rate_mean', 'zero_crossing_rate_std', 'zero_crossing_rate_min', 'zero_crossing_rate_max', 'zero_crossing_rate_skew', 'zero_crossing_rate_kurtosis', 'zero_crossing_rate_range',
        'labels']

**End of dropping data's section.**

## Load Data

1. Data will be loaded as .npy format, don't neccessarily need to change it to .csv

In [5]:
# You can start load data from here
# save to .npy after drop
df_q1_path = "dataset/data_afterDrop_q1.npy"
df_q2_path = "dataset/data_afterDrop_q2.npy"
df_q3_path = "dataset/data_afterDrop_q3.npy"
df_q4_path = "dataset/data_afterDrop_q4.npy"

# np.load
mod_data_q1 = np.load(df_q1_path, allow_pickle=True)
mod_data_q2 = np.load(df_q2_path, allow_pickle=True)
mod_data_q3 = np.load(df_q3_path, allow_pickle=True)
mod_data_q4 = np.load(df_q4_path, allow_pickle=True)

# we have below columns
cols = ['tempo', 'chroma_stft_mean', 'chroma_stft_std', 'chroma_stft_sum', 'chroma_stft_skew', 'chroma_stft_kurtosis',
        'chroma_cqt_mean', 'chroma_cqt_std', 'chroma_cqt_sum', 'chroma_cqt_skew', 'chroma_cqt_kurtosis',
        'chroma_vqt_mean', 'chroma_vqt_std', 'chroma_vqt_sum', 'chroma_vqt_skew', 'chroma_vqt_kurtosis',
        'melspectrogram_mean', 'melspectrogram_std', 'melspectrogram_min', 'melspectrogram_max', 'melspectrogram_sum', 'melspectrogram_skew', 'melspectrogram_kurtosis',
        'mfcc_mean', 'mfcc_std', 'mfcc_min', 'mfcc_max', 'mfcc_skew', 'mfcc_kurtosis', 'mfcc_median',
        'rms_mean', 'rms_std', 'rms_min', 'rms_max', 'rms_skew', 'rms_kurtosis',
        'spectral_centroid_mean', 'spectral_centroid_std', 'spectral_centroid_min', 'spectral_centroid_max', 'spectral_centroid_skew', 'spectral_centroid_kurtosis',
        'spectral_bandwidth_mean', 'spectral_bandwidth_std', 'spectral_bandwidth_min', 'spectral_bandwidth_max', 'spectral_bandwidth_skew', 'spectral_bandwidth_kurtosis',
        'spectral_contrast_mean', 'spectral_contrast_std', 'spectral_contrast_min', 'spectral_contrast_max', 'spectral_contrast_skew', 'spectral_contrast_kurtosis', 'spectral_contrast_range',
        'spectral_flatness_mean', 'spectral_flatness_std', 'spectral_flatness_min', 'spectral_flatness_max', 'spectral_flatness_skew', 'spectral_flatness_kurtosis', 'spectral_flatness_range',
        'spectral_rolloff_mean', 'spectral_rolloff_std', 'spectral_rolloff_min', 'spectral_rolloff_max', 'spectral_rolloff_skew', 'spectral_rolloff_kurtosis', 'spectral_rolloff_range',
        'tonnetz_mean', 'tonnetz_std', 'tonnetz_min', 'tonnetz_max', 'tonnetz_skew', 'tonnetz_kurtosis', 'tonnetz_range',
        'zero_crossing_rate_mean', 'zero_crossing_rate_std', 'zero_crossing_rate_min', 'zero_crossing_rate_max', 'zero_crossing_rate_skew', 'zero_crossing_rate_kurtosis', 'zero_crossing_rate_range',
        'labels']

In [10]:
mod_data_q1.shape

(225, 84)

## Check padding **No need to run! Already checked.**

No need to do padding after checking.

In [ ]:
# Initialize a list to hold the shape of the data for each feature
feature_shapes_q1 = []
feature_shapes_q2 = []
feature_shapes_q3 = []
feature_shapes_q4 = []

# Assuming you have complex structures and data is loaded such that it supports iteration over features
for feature_index in range(mod_data_q1.shape[1]):  # Iterate over features
    # Extract all samples for this feature
    samples = mod_data_q1[:, feature_index]

    # Check if all elements in this feature have the same shape
    shapes = set([np.array(sample).shape for sample in samples])

    # Store the unique shapes found for this feature
    feature_shapes_q1.append((feature_index, shapes))

for feature_index in range(mod_data_q2.shape[1]):  # Iterate over features
    # Extract all samples for this feature
    samples = mod_data_q2[:, feature_index]

    # Check if all elements in this feature have the same shape
    shapes = set([np.array(sample).shape for sample in samples])

    # Store the unique shapes found for this feature
    feature_shapes_q2.append((feature_index, shapes))

for feature_index in range(mod_data_q3.shape[1]):  # Iterate over features
    # Extract all samples for this feature
    samples = mod_data_q3[:, feature_index]

    # Check if all elements in this feature have the same shape
    shapes = set([np.array(sample).shape for sample in samples])

    # Store the unique shapes found for this feature
    feature_shapes_q3.append((feature_index, shapes))

for feature_index in range(mod_data_q4.shape[1]):  # Iterate over features
    # Extract all samples for this feature
    samples = mod_data_q4[:, feature_index]

    # Check if all elements in this feature have the same shape
    shapes = set([np.array(sample).shape for sample in samples])

    # Store the unique shapes found for this feature
    feature_shapes_q4.append((feature_index, shapes))

# Review the shapes for each feature
for feature_index, shapes in feature_shapes_q1:
    if len(shapes) > 1:
        print(f"Q1 Feature at index {feature_index} has varying shapes: {shapes}")
    # else:
    #     print(f"Feature at index {feature_index} has uniform shape: {shapes}")

for feature_index, shapes in feature_shapes_q2:
    if len(shapes) > 1:
        print(f"Q2 Feature at index {feature_index} has varying shapes: {shapes}")

for feature_index, shapes in feature_shapes_q3:
    if len(shapes) > 1:
        print(f"Q3 Feature at index {feature_index} has varying shapes: {shapes}")

for feature_index, shapes in feature_shapes_q4:
    if len(shapes) > 1:
        print(f"Q4 Feature at index {feature_index} has varying shapes: {shapes}")

The above cell checked for all q1, q2, q3, q4, for each feature, across 225 data the shape are the same.

In [ ]:
# decide whether need padding
# Example for a list of arrays (if `data` is a list or an array of arrays)
lengths_q1 = [seq.shape[0] for seq in mod_data_q1]  # Assuming sequences
                                               # are the first dimension
lengths_q2 = [seq.shape[0] for seq in mod_data_q2]
lengths_q3 = [seq.shape[0] for seq in mod_data_q3]
lengths_q4 = [seq.shape[0] for seq in mod_data_q4]

# Check if all sequences have the same length
if len(set(lengths_q1)) > 1:
    print("Q1 Padding needed. Sequence lengths vary.")
else:
    print("Q1 No padding needed. All sequences have the same length.")
# Check if all sequences have the same length
if len(set(lengths_q2)) > 1:
    print("Q2 Padding needed. Sequence lengths vary.")
else:
    print("Q2 No padding needed. All sequences have the same length.")
# Check if all sequences have the same length
if len(set(lengths_q3)) > 1:
    print("Q3 Padding needed. Sequence lengths vary.")
else:
    print("Q3 No padding needed. All sequences have the same length.")
# Check if all sequences have the same length
if len(set(lengths_q4)) > 1:
    print("Q4 Padding needed. Sequence lengths vary.")
else:
    print("Q4 No padding needed. All sequences have the same length.")

Q1 No padding needed. All sequences have the same length.
Q2 No padding needed. All sequences have the same length.
Q3 No padding needed. All sequences have the same length.
Q4 No padding needed. All sequences have the same length.


## Transfer loaded .npy into tensor.

In [6]:
# TODO By 张北
#融合四个q
np_whole=np.vstack((np.vstack((np.vstack((mod_data_q1,mod_data_q2)),mod_data_q3)),mod_data_q4))
df_whole = pd.DataFrame(np_whole,columns=cols)

In [7]:
df_whole.head()

,tempo,chroma_stft_mean,chroma_stft_std,chroma_stft_sum,chroma_stft_skew,chroma_stft_kurtosis,chroma_cqt_mean,chroma_cqt_std,chroma_cqt_sum,chroma_cqt_skew,...,tonnetz_kurtosis,tonnetz_range,zero_crossing_rate_mean,zero_crossing_rate_std,zero_crossing_rate_min,zero_crossing_rate_max,zero_crossing_rate_skew,zero_crossing_rate_kurtosis,zero_crossing_rate_range,labels
0,[129.19921875],"[0.3402438, 0.4108848, 0.32207942, 0.43924117,...","[0.25901443, 0.34396023, 0.24817161, 0.3390962...",5754.600098,"[0.99369633, 0.6889807, 1.1558719, 0.51363957,...","[0.18223047, -1.0339843, 0.7369144, -1.18541, ...","[0.37230334, 0.59613794, 0.349376, 0.5221062, ...","[0.1650009, 0.28621086, 0.17438404, 0.24190375...",7482.439453,"[1.1793867, 0.19034995, 1.4648391, 0.5109394, ...",...,"[-0.2676932002114798, -0.2611242959606561, -0....","[0.39503816324089674, 0.44382569880070666, 0.6...",[0.1439822635135135],[0.06052925895431856],[0.03125],[0.564453125],[2.299913592613744],[9.972581240443636],[0.533203125],1
1,[161.4990234375],"[0.36104006, 0.2428113, 0.33913192, 0.41598046...","[0.3057084, 0.21803722, 0.26099977, 0.31606582...",5868.901367,"[0.8817804, 1.4171929, 0.9687368, 0.77790874, ...","[-0.35459352, 1.3924265, 0.051932335, -0.83470...","[0.39986077, 0.30028376, 0.41042277, 0.5339542...","[0.2795176, 0.11410666, 0.1324839, 0.23995899,...",6873.734375,"[1.3378943, 1.0632907, 0.9806561, 0.91683173, ...",...,"[-0.4546138798763306, -0.5757695864820596, 0.0...","[0.42942833816117487, 0.4362426048761109, 0.52...",[0.10062665902509653],[0.06195558099185238],[0.0126953125],[0.53857421875],[2.4103148531571437],[10.632752706921927],[0.52587890625],1
2,[99.38401442307692],"[0.3350637, 0.4391525, 0.3131803, 0.36769086, ...","[0.23067027, 0.34413788, 0.2166038, 0.3000164,...",5885.837891,"[0.63521445, 0.6002824, 0.6111119, 0.9701896, ...","[-0.44197345, -1.1731204, -0.6007376, -0.27134...","[0.2838195, 0.55256397, 0.23260526, 0.35447538...","[0.13484542, 0.31740782, 0.123309836, 0.267685...",6406.210449,"[1.078106, 0.44249153, 1.7751677, 1.425022, 1....",...,"[0.07777779783759664, -0.66986980775311, -0.57...","[0.549337563990827, 0.45787447752394766, 0.692...",[0.06854375301640926],[0.03186446827069103],[0.01806640625],[0.21875],[1.0653329036995505],[1.313176804637732],[0.20068359375],1
3,[123.046875],"[0.20232584, 0.4100135, 0.2596639, 0.14443561,...","[0.15172842, 0.36189678, 0.26187456, 0.1126129...",4598.774902,"[1.5267997, 0.74306566, 1.7494311, 1.5250498, ...","[3.3818812, -1.0821086, 2.2402334, 3.302959, 1...","[0.19787204, 0.38152912, 0.2595981, 0.18471201...","[0.11222625, 0.2533469, 0.15406808, 0.10194606...",4871.200684,"[1.6630076, 1.2764502, 1.5215753, 1.4900329, 1...",...,"[-0.689955821707545, 0.011593890255965356, -0....","[0.6551601962522167, 0.8039702548013667, 0.817...",[0.0696255127895753],[0.016080018069265088],[0.02978515625],[0.1484375],[0.7492532964882583],[1.9720264141033086],[0.11865234375],1
4,[103.359375],"[0.20571218, 0.18657087, 0.284147, 0.23309906,...","[0.26805663, 0.24865834, 0.31335503, 0.2693379...",3841.712158,"[1.8727081, 1.9668415, 1.3176929, 1.7174534, 1...","[2.6374311, 3.2114024, 0.471771, 2.033866, 0.9...","[0.4677634, 0.34280092, 0.41840968, 0.42241988...","[0.29720858, 0.25228196, 0.27344728, 0.2717988...",6369.010254,"[0.54591733, 1.2141986, 0.8864644, 0.9038617, ...",...,"[2.0776852407507347, 1.3959966015028629, 0.106...","[0.9353934797999381, 0.8043349604683768, 1.031...",[0.09968026061776061],[0.039801127600770446],[0.025390625],[0.23291015625],[0.24549962817541723],[-0.5180193190257767],[0.20751953125],1


In [8]:
# TODO
# demo_x - x prepared for training NN
# demo_y - labels for NN predicted output to compare
demo_x = df_whole[[col for col in df_whole.columns if col != 'labels']]
demo_y = df_whole["labels"]
tensor_list = torch.tensor(np.stack(demo_x['tempo'].values), dtype=torch.float)
for i in cols[1:-1]:
  #设置报错函数 如果是类似sum的格式，无法用第一个的格式，就走except里面的，问题，为什么只有四个sum，其他数据的sum没有写吗
  #答，因为其他数据sum之后没有意义，所以没有写sum
  try:
    current_tensor = torch.tensor(np.stack(demo_x[i].values), dtype=torch.float)
    tensor_list = torch.cat((tensor_list, current_tensor), dim=1)
  except:
    current_tensor_numeric = pd.to_numeric(demo_x[i], errors='coerce')
    current_tensor_numpy = np.array(current_tensor_numeric, dtype=float)
    current_tensor = torch.tensor(current_tensor_numpy, dtype=torch.float)
    demo_sum_unsqez = current_tensor.unsqueeze(1)
    tensor_list = torch.cat((tensor_list, demo_sum_unsqez), dim=1)

print(f"demo_x.shape: {tensor_list.shape}")


demo_y_num = pd.to_numeric(demo_y, errors='coerce')
demo_y_array = np.array(demo_y_num, dtype=float)
demo_y = torch.tensor(demo_y_array, dtype=torch.float)
demo_y = demo_y - 1  # Shift labels from 1, 2, 3, 4 to 0, 1, 2, 3
print(f"demo_y.shape: {demo_y.shape}")

demo_x.shape: torch.Size([900, 1187])
demo_y.shape: torch.Size([900])


## Demo train on NN (**You don't need to run these cells**)

**TODO** **====**   

!!! Need to finally view whether needs to pad data(whether they have same dimension within the same feature) !!!

  _solved_: Already checked, no need to pad. 2024.03.05 Xiyah

!!! They are all run based on **pandas.DataFrame**. Now we have all data loaded as .npy, if in practical, need to figure out how to use .npy to direcly transfer to tensor. !!!

!!! It is important to notice that, while spliting train and test data, needs to normalize train data first based on each features (do feature normalization) and the store the mean and variance results, so that we can use them(the train data mean and variance) while normalizing test data. !!!

**====**

We want to see whether the input containing both 1d vector and scalar can be trained in a NN.

In [ ]:
### Xiyah ###
# simple example to see whether NN can train input containing
# both 1d vector and scalar
demo_data = df_q1[["chroma_stft_mean", "chroma_stft_sum", "labels"]]
demo_data.head()
### Xiyah ###

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# split x and y for demo
# x: anything except "labels"
# y: "labels": {1, 2, 3, 4}
demo_x = demo_data[[col for col in demo_data.columns if col != 'labels']]
demo_y = demo_data["labels"]
#demo_x.head()

In [ ]:
### Xiyah ###
# Adapt inputs to tensor

## Convert chroma_stft_mean to a tensor
# Stack the list of np.arrays to create a single np.array of shape (225, 12)
demo_stft_mean_tensor = torch.tensor(np.stack(demo_x['chroma_stft_mean'].values), dtype=torch.float)

## Convert chroma_stft_sum to a tensor
# Since these are scalar values, we can directly convert the column to a np.array, then to a tensor (225,)
# Convert 'chroma_stft_sum' to a numeric type
chroma_stft_sum_numeric = pd.to_numeric(demo_x['chroma_stft_sum'], errors='coerce')

# Now convert to a NumPy array, ensuring a numeric dtype
chroma_stft_sum_numpy = np.array(chroma_stft_sum_numeric, dtype=float)

# Finally, convert this to a PyTorch tensor
demo_stft_sum_tensor = torch.tensor(chroma_stft_sum_numpy, dtype=torch.float)
#demo_stft_sum_tensor = torch.tensor(demo_x['chroma_stft_sum'].values, dtype=torch.float)

## Unsqueezing chroma_stft_sum_tensor to make its shape (225, 1) for concatenation
demo_sum_unsqez = demo_stft_sum_tensor.unsqueeze(1)

## Concatenate along the second dimension to get a unified tensor of shape (225, 13)
# this is X
unified_tensor = torch.cat((demo_stft_mean_tensor, demo_sum_unsqez), dim=1)

unified_tensor.shape

## Convert labels to tensor
# this is Y - shape (225,)
demo_y_num = pd.to_numeric(demo_y, errors='coerce')
demo_y_array = np.array(demo_y_num, dtype=float)
demo_y = torch.tensor(demo_y_array, dtype=torch.float)
# Finished converting both inputs and labels into tensor
### Xiyah ###

torch.Size([225, 13])

In [ ]:
# important
demo_y = demo_y - 1  # Shift labels from 1, 2, 3, 4 to 0, 1, 2, 3

In [ ]:
demo_y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# built a simple NN
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1187, 128)  # Input layer with 13 features
        self.fc2 = nn.Linear(128, 64)  # Hidden layer
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 4)   # Output layer with 4 outputs for 4 classes

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Activation function for hidden layers
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # No activation function here, nn.CrossEntropyLoss() will apply softmax
        return x

In [ ]:
# run model
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # Cross Entropy Loss for multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [ ]:
# Simple example using only forward pass
labels = demo_y.long()  # Ensure labels are of type torch.long

# Forward pass
outputs = model(tensor_list)

# Compute Loss
loss = criterion(outputs, labels)

print("Loss:", loss.item())

Loss: nan


In [ ]:
outputs

tensor([[ 7.4081e+02,  1.0533e+02,  1.7182e+00, -1.0368e+03],
        [ 3.5262e+03,  4.7966e+02, -1.0303e+01, -4.7296e+03],
        [ 2.0975e+03,  2.8689e+02, -5.4511e+00, -2.8098e+03],
        ...,
        [ 5.4489e+02,  8.3613e+01, -5.7192e+00, -7.5185e+02],
        [ 3.0893e+02,  4.4940e+01, -2.1656e+00, -4.2538e+02],
        [ 9.7906e+02,  1.1161e+02,  4.9008e+00, -1.3132e+03]],
       grad_fn=<AddmmBackward0>)

In [ ]:
unified_tensor.shape

torch.Size([225, 13])

In [ ]:
# print(outputs)
probabilities = torch.softmax(outputs, dim=1)
#print(probabilities)
_, predicted_classes = torch.max(probabilities, 1)
print(predicted_classes)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
### Haoruo ###
#Data cleaning mean body
df_q1_after=pd.DataFrame()
#For chroma_stft

##For chroma_stft_origin 我们需要填补空缺值 (225, 12, 1295)
data = df_q1['chroma_stft_demos'].values.reshape((225, 1))
data_sequences = []
sequences = []
for i in data:
  sequences.append(i[0])
max_len = max(s.shape[1] for s in sequences)
padded_sequences_all = []
for seq in sequences:
  padded_sequences = []
  for s in seq:
    padded_sequences.append(np.append(s,np.zeros((max_len-len(s),))))
  padded_sequences_all.append(np.array(padded_sequences))
np.array(padded_sequences_all).shape
# print(df_q1_after["chroma_stft"].shape)
# ##For chroma_stft_mean
# sequences = []
# for i in df_q1['chroma_stft_mean'].values.reshape((225,1)):
#   sequences.append(i[0])
# df_q1_after["chroma_stft_mean"]=np.array(sequences).reshape((225, 12,1))

# ##For chroma_stft_std
# sequences = []
# for i in df_q1['chroma_stft_std'].values.reshape((225,1)):
#   sequences.append(i[0])
# df_q1_after["chroma_stft_std"]=np.array(sequences).reshape((225, 12,1))

# ##For chroma_stft_skew
# sequences = []
# for i in df_q1['chroma_stft_skew'].values.reshape((225,1)):
#   sequences.append(i[0])
# df_q1_after["chroma_stft_skew"]=np.array(sequences).reshape((225, 12,1))

# ##For chroma_stft_kurtosis
# sequences = []
# for i in df_q1['chroma_stft_kurtosis'].values.reshape((225,1)):
#   sequences.append(i[0])
# df_q1_after["chroma_stft_kurtosis"]=np.array(sequences).reshape((225, 12,1))

### Haoruo ###

(225, 12, 1297)

In [ ]:
sequences = []
for i in df_q1['chroma_stft_mean'].values.reshape((225,1)):
  sequences.append(i[0])
print(np.array(sequences).reshape((225, 12,1)))
# print(df_q1['chroma_stft_std'].shape)
# print(df_q1['chroma_stft_skew'].shape)
# print(df_q1['chroma_stft_kurtosis'].shape)

[[0.36104006]
 [0.2428113 ]
 [0.33913192]
 [0.41598046]
 [0.2915763 ]
 [0.30554944]
 [0.28725767]
 [0.39951402]
 [0.43674076]
 [0.42523807]
 [0.5881421 ]
 [0.4389887 ]]


# Algorithm Implementation
Xiangyu: The following is a implementation of XLNet as a base model (inspired by article https://link.springer.com/chapter/10.1007/978-3-030-72240-1_12)

In [ ]:
###Xiangyu###
# transformer based model
class multitask_model(nn.Module):
    def __init__(self, transformer):
        super(multitask_model, self).__init__()
        self.transformer = transformer
        self.fc_middle = nn.Sequential(
            nn.ReLU(),
            nn.Linear(13, 64),
            nn.Dropout(0.25)
        )
        self.fc_quadrant = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.Linear(32, 4),
        )
    def forward(self, b_input_ids, token_type_ids=None, attention_mask=None, labels=None):
        outputs = self.transformer(b_input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
        output = self.fc_middle(outputs[1])
        fc_quadrant_out = self.fc_quadrant(output)
        return fc_quadrant_out

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(demo_x, demo_y, test_size=0.2, random_state=42)
X_train = torch.from_numpy(X_train).float()
X_test= torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

TypeError: expected np.ndarray (got DataFrame)

In [ ]:
###Xiangyu###
# train and evaluation
def train(i):
    model.train()
    total_loss = 0.0
    total_predicted_label = np.array([])
    total_actual_label = np.array([])
    train_len = 0
    f_acc = 0

    for step, batch in enumerate(train_loader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        if b_labels.size(0) <= 1:
            continue


        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        pred = outputs[1].detach().cpu().numpy()
        batch_f_acc, pred_flat = flat_accuracy(pred, b_labels)
        f_acc += batch_f_acc
        loss = outputs[0]
        loss.sum().backward()
        optimizer.step()


        labels_flat = b_labels.flatten().cpu().detach().numpy()
        total_actual_label = np.concatenate((total_actual_label, labels_flat))
        total_predicted_label = np.concatenate((total_predicted_label, pred_flat))

        total_loss += outputs[0].sum()
        train_len += b_input_ids.size(0)

        if step%100 == 0 and step:
            precision, recall, f1_measure, _ =  precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
            logging.info("Train: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %(train_len*100.0/train_inputs.size(0), i, step,total_loss/train_len, f_acc*100.0/train_len,precision*100., recall*100., f1_measure*100.))

        # if torch.cuda.device_count() > 1:
        #     p = 100
        #     path = save_model_path + '/e_' + str(i) + "_" + str(p) + ".ckpt"
        #     torch.save(model.module.state_dict(), path)
        # else:
        #     torch.save(model.state_dict(), path)

    precision, recall, f1_measure, _ = precision_recall_fscore_support(total_actual_label, total_predicted_label, average='macro')
    logging.info("Train: %5.1f\tEpoch: %d\tIter: %d\tLoss: %5.5f\tAcc= %5.3f\tPrecision= %5.3f\tRecall= %5.3f\tF1_score= %5.3f" %(train_len*100.0/train_inputs.size(0), i, step,total_loss/train_len, f_acc*100.0/train_len, precision*100., recall*100., f1_measure*100.))


# Updates
